In [27]:
from numpy import *
U,Sigma,YT = linalg.svd([[1,1],[7,7]])

In [28]:
U

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [29]:
Sigma

array([  1.00000000e+01,   2.82797782e-16])

In [30]:
YT

array([[-0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678]])

In [31]:
def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]

In [32]:
U1,Sigma1,VT1 = linalg.svd(loadExData())
Sigma1

array([  9.64365076e+00,   5.29150262e+00,   6.51609210e-16,
         2.14818942e-16,   5.18511491e-17])

In [33]:
Sig3 = mat([[Sigma1[0],0,0],[0,Sigma1[1],0]])
U1[:,:2]*Sig3*VT1[:3,:]

matrix([[ -3.37368515e-18,   9.15076142e-16,   9.15076142e-16,
           2.00000000e+00,   2.00000000e+00],
        [ -9.18449827e-16,   4.59224914e-16,   4.59224914e-16,
           3.00000000e+00,   3.00000000e+00],
        [ -3.06149942e-16,   1.53074971e-16,   1.53074971e-16,
           1.00000000e+00,   1.00000000e+00],
        [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          -7.22134412e-17,  -7.22134412e-17],
        [  2.00000000e+00,   2.00000000e+00,   2.00000000e+00,
          -1.13379157e-16,  -1.13379157e-16],
        [  5.00000000e+00,   5.00000000e+00,   5.00000000e+00,
           4.44172017e-17,   4.44172017e-17],
        [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          -5.66895787e-17,  -5.66895787e-17]])

In [34]:
from numpy import linalg as la

#采用距离计算相似度
def ecludSim(inA,inB):
    return 1.0/(1.0+la.norm(inA-inB))

#皮尔逊相关系数法
def pearsSim(inA,inB):
    if(len(inA)<3): return 1
    return 0.5+0.5*corrcoef(inA,inB,rowvar=0)[0][1]

#余弦值法
def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [35]:
myMat = mat(loadExData())
ecludSim(myMat[:,0],myMat[:,4])

0.12973190755680383

In [36]:
ecludSim(myMat[:,0],myMat[:,0])

1.0

In [37]:
pearsSim(myMat[:,0],myMat[:,4])

0.20596538173840329

In [38]:
pearsSim(myMat[:,0],myMat[:,0])

1.0

In [39]:
cosSim(myMat[:,0],myMat[:,4])

0.5

In [40]:
cosSim(myMat[:,0],myMat[:,0])

1.0

In [41]:
#基于物品相似度推荐引擎
#计算在给定相似度就按方法的条件下，用户对于物品的估计评分值
def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item].A>0,dataMat[:,j].A>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item],dataMat[overLap,j])#计算相似度
        print('the %d and %d similarity is: %f' % (item, j, similarity))#对于给出的物品，哪个物品和它的相似度是多少
        simTotal += similarity#相似度和
        ratSimTotal += similarity * userRating#得分为，每一个的相似度和遍历的那个物品评分相乘
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal#最后返回得分除以总的相似度

#推荐引擎
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1]#得到没有评分的菜
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)#
        itemScores.append((item, estimatedScore))#得到菜单得分集合
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]
    

In [42]:
myMat = mat(loadExData())
#做少量的更改
myMat[0,1] = myMat[0,0]=myMat[1,0]=myMat[2,0]=4
myMat[3,3]=2
myMat

matrix([[4, 4, 0, 2, 2],
        [4, 0, 0, 3, 3],
        [4, 0, 0, 1, 1],
        [1, 1, 1, 2, 0],
        [2, 2, 2, 0, 0],
        [5, 5, 5, 0, 0],
        [1, 1, 1, 0, 0]])

In [43]:
recommend(myMat,2)

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 0.928746
the 1 and 4 similarity is: 1.000000
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 1.000000
the 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0243290220056256)]

In [44]:
#使用距离相似度计算
recommend(myMat,2,simMeas=ecludSim)

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 0.309017
the 1 and 4 similarity is: 0.333333
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 0.500000
the 2 and 4 similarity is: 0.000000


[(2, 3.0), (1, 2.8266504712098603)]

In [45]:
#使用皮尔逊相关系数法计算
recommend(myMat,2,simMeas=pearsSim)

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 1.000000
the 1 and 4 similarity is: 1.000000
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 1.000000
the 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0)]

In [46]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [50]:
U2,Sigma2,VT2 = la.svd(mat(loadExData2()))
Sigma2

array([ 15.77075346,  11.40670395,  11.03044558,   4.84639758,
         3.09292055,   2.58097379,   1.00413543,   0.72817072,
         0.43800353,   0.22082113,   0.07367823])

In [53]:
Sig2 = Sigma2*2
sum(Sig2)*0.9

92.147407101645186

In [55]:
sum(Sig2[:4])

86.108601120819117

In [57]:
#计算相似度中加入SVD思想
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    U,Sigma,VT = la.svd(dataMat)
    Sig4 = mat(eye(4)*Sigma[:4]) #建立对角矩阵只要包含90%能量的奇异值
    xformedItems = dataMat.T * U[:,:4] * Sig4.I  #构建转换后的物品
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,xformedItems[j,:].T)
        print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [58]:
recommend(myMat,1,estMethod=svdEst)

the 1 and 0 similarity is: 0.498142
the 1 and 3 similarity is: 0.498131
the 1 and 4 similarity is: 0.509974
the 2 and 0 similarity is: 0.552670
the 2 and 3 similarity is: 0.552976
the 2 and 4 similarity is: 0.217301


[(2, 3.4177569186592378), (1, 3.3307171545585641)]

In [59]:
recommend(myMat,1,estMethod=svdEst,simMeas=pearsSim)

the 1 and 0 similarity is: 0.280908
the 1 and 3 similarity is: 0.948940
the 1 and 4 similarity is: 0.606039
the 2 and 0 similarity is: 0.556905
the 2 and 3 similarity is: 0.552447
the 2 and 4 similarity is: 0.214974


[(2, 3.4205197987866822), (1, 3.1530093683701286)]

In [78]:
#将数字矩阵画出来
def printMat(inMat,thresh=0.8):
    m,n = shape(inMat)
    for i in range(m):
        for j in range(n):
            if float(inMat[i,j])>thresh:
                print(1,end=''),
            else:print(0,end=''),#在python3中默认自动换行，设置end=''的时候为不换行
        print("")
        
def imgCompress(numSV=3,thresh=0.8):
    myl = []
    for line in open('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch14/0_5.txt').readlines():
        newRow=[]
        for i in range(32):
             newRow.append(int(line[i]))
        myl.append(newRow)
    myMat = mat(myl)
    print("****original matrix******")
    printMat(myMat, thresh)
    U,Sigma,VT = la.svd(myMat)
    SigRecon = mat(zeros((numSV, numSV)))
    for k in range(numSV):#构建奇异值组成的对角矩阵
        SigRecon[k,k] = Sigma[k]
    reconMat = U[:,:numSV]*SigRecon*VT[:numSV,:]
    print("****reconstructed matrix using %d singular values******" % numSV)
    printMat(reconMat, thresh)

In [79]:
#取前两个奇异值进行图像重构
imgCompress(2)

****original matrix******
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
00000000011111111111111110000000
00000000001111111111111110000000
00000000001111111111111110000000
00000000000111111